# Setup Intelligent Recommendations Service

Intelligent Recommendations is a general purpose service that offers patented capabilities to more effectively drive desired outcomes out of the box such as “shop similar looks,” “shop by description,” “real time,” “session based”, Item based recommendations that can combine User interactions and Item Metadata. Businesses can promote and personalize any content type, such as sellable products, consumable media, documents, videos, and more.

Learn more at [Explore Intelligent Recommendations](https://intelligent-recommendations.microsoft.com/).


## Prerequisites

If you have completed all the steps from the *01_Data_Preparation.ipynb* notebook you have a dataset ready for the Intelligent Recommendations service to consume, go on to "Create & Configure Intelligent Recommendations". 

If you didn't complete the data preparation first, you can setup the data files by completing the **Prerequisite, Step 1 and Step 2** from the [Quick Start Guide](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/quickstart#prerequisites-get-azure-subscription-and-tenant-information-ready) using the zip file from this repo - **groceries_IRprepared.zip**.  

## 1. Create & Configure Intelligent Recommendations

You are now ready to setup the Intelligent Recommendation Service!

**1.1 Create Service Account**

Follow steps 1-5 in [Create the service account](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/deploy-intelligent-recommendations-account#create-the-service-account).  Leave the "Reports Connection String" blank at this point, we will configure it in the next step once the resource is created.

**TIP:** Do add your tenant and user id as described in the Note on Step 4!  You can find your Tenant ID from the Azure Active Directory area of the Azure Portal Home page.  Then, click on Users, and search for yourself to find your Object ID, which is your User ID.  Make sure you select "User" as the Principal Type.

**1.2 Configure Access to Storage Account **

Setup access for your IR Service Account to the storage container with the files setup in the previous notebook (01_Data_Preparation) or from the prepared files (Prerequisite step), follow the steps in [Setup security for the container](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/deploy-data-lake-storage#set-up-security-for-the-container) and [Verify your role assignments](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/deploy-data-lake-storage#verify-your-role-assignments)

**1.3 Configure Logging**

Configure logging using the folder you created in your Azure Storage Container for logs by following the instructions [here](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/setup-error-logging#configure-the-reports-feature).  Note you will link to the ir_logs folder, not the root.  

**TIP:** For our example, the connection string should be similar to (replace *your_storage_account* with your account name): 
&nbsp;&nbsp;&nbsp;&nbsp;*https://your_storage_account.blob.core.windows.net/ircontainer/ir_logs*  

**1.4 Add Modeling & Serving Components**

Complete all the steps in the [Add modeling and serving components](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/deploy-intelligent-recommendations-account#add-modeling-and-serving-components) section.

**TIP:** For the serving component, you can select the minimum number of requests per second (1) for the "Pre-allocated Capacity" to work through this example.

**TIP:** When you create the Modeling component there is an option to select "Feature Set".  We show examples of using the "Premium" features in this notebook, but if you want to maintain a free/lower tier cost for the service, you can select "Basic" or "Standard". 


Once the modeling component is deployed, it will automatically initiate the cooking process that processes the data and creates the recommendations.  This may take anywhere from a few minutes to a few hours.  The best way to monitor progress is to watch for files to be written to the logging folder (e.g. ir_logs in this example).

When the cooking process completes you can view the results at the [Results Explorer Portal](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/results-explorer#step-2-view-results).   Until the process is complete though, you will not see any results when you "Execute Request".

While IR processes, continue with the next steps so you can authenticate and make API calls to the service once it's complete.


## 2. Create Client Application & Configure Authentication

Intelligent Recommendations requires an OAuth2 authentication. Determine (or create) the Active Directory application from which to call the service endpoint with authentication. Any unauthenticated calls will fail with an error.  

To establish this, complete the tasks in [Step 2: Setup Client Application](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/quickstart-endpoint#step-2-set-up-a-client-application).

Go back to the application you registered in the previous step, continue with [Step 3: Configure Authentication](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/quickstart-endpoint#step-3-configure-authentication).  You can use either a secret or a certificate for authentication, for this example we'll create a secret.

**TIP:** Make sure to note the "Value" for your Client Secret in the previous step.  If you lose it, you may not be able to access it again, in that case add a new client Secret and note the Value before navigating from the page.

You can stop once you get to the "Construct API Request" step as we will do this in this Notebook.




## 3. (Optional) Configure CORS

While this is not required for this example solution accelerator, if you wanted to use a web application in another domain to access the Intelligent Recommendations service, you could configure Cross-Origin Resource Sharing.

Instructions provided [here](https://docs.microsoft.com/en-us/industry/retail/intelligent-recommendations/deploy-intelligent-recommendations-account#configure-cors-optional).

# Model Processing

After the modeling resource has been created and deployed in the previous steps, Intelligent Recommendations service will continue to process data through a periodic cooking cycle. By default, the data cooking cycle occurs every 36 hours.

We will manually trigger the cooking cycle to initiate our model processing in the next section.

In [1]:
import requests
import json
import base64
import pandas as pd

The following fields should be completed based on the Intelligent Recommendations account and the Client Application setup in the previous steps.

In [2]:
tenant_id = 'Your_Azure_Tenant_Id'

ir_base_uri = 'Your_IR_Service_Endpoint'  #the Url from the overview page for the Service Endpoint in the IR account (Step 1 above)
model = 'Your_Model_Name'  #the name of the modeling component in your IR account

client_id = 'Your_Client_ID'  #the application client id you created in Step 2 of the setup.
client_secret = 'Your_Client_Secret_Value' #from step 2 above.

auth_server_url = "https://login.microsoftonline.com/" + tenant_id + "/oauth2/v2.0/token"
scope = ir_base_uri + '/.default'

Prepare the token we will use for authorizated calls to the API.

In [29]:
# Encode the client ID and client secret
authorization = base64.b64encode(bytes(client_id + ":" + client_secret, "ISO-8859-1")).decode("ascii")

headers = {
    "Authorization": f"Basic {authorization}",
    "Content-Type": "application/x-www-form-urlencoded"
}
body = {
    "grant_type": "client_credentials", "scope" : scope
}

response = requests.post(auth_server_url, data=body, headers=headers)
#print(response.text)


tokens = json.loads(response.text)
token = tokens['access_token']
#print(token)

## Trigger Modeling Manually

This section is purposefully commented, to avoid triggering the modeling process unintentionally.  When you need to trigger modeling, keep in mind that there are limitations on this operation, for more details see [Limitations with manually triggering the modeling process](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/trigger-modeling#limitations-with-manually-triggering-the-modeling-process).

If you have just setup a new model, you will want to uncomment and run the following to confirm that the modeling process is running, or to start it if it didn't begin automatically.

In [31]:
ir_trigger_model = ir_base_uri + '/Control/V1.0/Model?modeling={}'.format(model)

### Uncomment below to execute the request
# api_call_headers = {'Authorization': 'Bearer ' + token}
# api_call_response = requests.post(ir_trigger_model, headers=api_call_headers)
# print(api_call_response.text)

{"error":{"code":"TooManyRequests","message":"Too many requests"}}


If the IR modeling process isn't running, you will get the message:

```console
{"status":"Success"}
```



If the process was running (or if you repeat/run again) you will get the message:

```console
{"error":{"code":"TooManyRequests","message":"Too many requests"}}
```

The processing will take time, check back every 10 minutes and run "Test API Call" (next section) until you get a list of popular items.  

You can also review the logs and reporting from the service, for more information see [Modeling status reports](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/modeling-status-reports) and  [Error logs](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/error-logs).

You can go on to the next topic/notebook and setup Personalizer while you wait, using the **03_Personalizer.ipynb** notebook.

## Test API Call

Our first call to the Intelligent Recommendations model will confirm connectivity and the status of the model processing.  If the model has completed processing it will return a set of "popular" items.  

The first time you run this after setting up the Intelligent Recommendations service and manually triggering the modeling process, the model will take time to process so you will likely see a status of "WaitingForData".

In [32]:
ir_popular_items = ir_base_uri + '/reco/v1.0/popular?modeling={}'.format(model)

api_call_headers = {'Authorization': 'Bearer ' + token}
api_call_response = requests.get(ir_popular_items, headers=api_call_headers)

print(api_call_response.text)

{"id":"Lists","name":"Lists","version":"v1.0","items":[{"id":"165","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"103","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"123","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"139","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"166","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"124","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"157","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"13","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"131","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"31","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"106","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"110","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"134","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"21","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"12","trackingId":"00000000-0000

If the processing has completed, you will get a list of popular items that should look similar to:

```console
{"id":"Lists","name":"Lists","version":"v1.0","items":[{"id":"165","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"103","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"123","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"139","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"166","trackingId":"00000000-0000-0000-0000-000000000003"},
...
{"id":"112","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"50","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"16","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"65","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"9","trackingId":"00000000-0000-0000-0000-000000000003"},{"id":"41","trackingId":"00000000-0000-0000-0000-000000000003"}],"title":"Popular","longTitle":"Popular","titleId":5,"pagingInfo":{"totalItems":167},"status":"Success"}
```

If instead, you get the following message:

```console
{"id":"Lists","name":"Lists","version":"v1.0","items":[],"title":"Popular","longTitle":"Popular","titleId":5,"pagingInfo":{"totalItems":0},"status":"WaitingForData","details":"The request data is currently unavailable. It may be cooked and become available soon."}
```

Your Intelligent Recommendations Service is likely still processing the data - you can also monitor the logs and reporting from the service.  For more information see [Modeling status reports](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/modeling-status-reports) and  [Error logs](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/error-logs).

If you have another return result than shown above, reference [API Status Codes](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/sample-api) for more information and troubleshooting tips.

# Recommendations Setup

The next section provides examples of the different API calls to the Intelligent Recommendations model and how the results can be applied.


Details about the modeling algorithms available can be found in the [Intelligent Recommendations Algorithms Overview] (https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/modeling-faq#intelligent-recommendations-algorithms-overview) and further guidance on selecting the right function for your business requirements is available in the [FAQ](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/modeling-faq#what-algorithm-and-list-type-should-i-use-for-my-business).

## Load Product Catalog and Orders Data

Once the IR Service has completed processing and you are able to view the list of popular items when you test the API call, proceed with the next steps.

We use the product catalog and orders data (from the 01_Data_Preparation.ipynb notebook) to help interpret the results from the Intelligent Recommendation system.  The files are also available in the repo, so you can continue working with those if you didn't complete the first section.

** Product Catalog **

In [7]:
products_df=pd.read_csv('data/products.csv')
print(products_df.shape)
products_df.head()

(167, 3)


,itemId,itemDescription,earliestDate
0,1,Instant food products,2014-01-01T00:00:00.000Z
1,2,UHT-milk,2014-01-02T00:00:00.000Z
2,3,abrasive cleaner,2014-01-21T00:00:00.000Z
3,4,artif. sweetener,2014-02-10T00:00:00.000Z
4,5,baby cosmetics,2014-01-09T00:00:00.000Z


** Orders (Interactions) **

In [8]:
orders_df =pd.read_csv('data/orders.csv')
print(orders_df.shape)
orders_df.head()

(38765, 5)


,basketId,Member_number,Date,itemId,itemDescription
0,1,1000,2014-06-24T00:00:00.000Z,106,pastry
1,1,1000,2014-06-24T00:00:00.000Z,129,salty snack
2,1,1000,2014-06-24T00:00:00.000Z,165,whole milk
3,2,1000,2015-03-15T00:00:00.000Z,131,sausage
4,2,1000,2015-03-15T00:00:00.000Z,133,semi-finished bread


## API Calls Functions

This function constructs the API calls for the different types of recommendations available in the service.  For more information and options see the [API Reference](https://learn.microsoft.com/en-us/rest/api/industry/intelligent-recommendations/). 

As you test these functions, reference [API Status Codes](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/sample-api) for more details and descriptions of the status returend.

Additional examples are available in the [Sample API Calls](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/sample-api) documentation.


In [9]:
def recommendation_uri_builder(base_uri, command, model, seed_item = 0, seed_user = 0, return_count = 100, seed_items_list = '', skip_count = 0):

    #as of July 2022, this is the API path
    ir_path = '/reco/v1.0/'

    # Get Similar Items - Returns a list of item-to-item recommendations. Recommended items can be similar to the seed item or related in some way, based on the way data is structured and the algorithm.
    # algo type for similar are Visual, Textual: '/reco/v1.0/similar/{}?AlgoType=Textual&modeling={}&count={}'.format(seed_item, model, return_count)
    if command == 'similar_text':  
        uri = ir_base_uri + ir_path + 'similar/{}?AlgoType=Textual&modeling={}&count={}'.format(seed_item, model, return_count)
    elif command == 'similar_visual':  
        uri = ir_base_uri + ir_path + 'similar/{}?AlgoType=Visual&modeling={}&count={}'.format(seed_item, model, return_count)
   
    # Popular Items - Returns the most popular items from the catalog, ranked by the number of interactions over a period of time.
    elif command == 'popular':
        uri = ir_base_uri + ir_path + 'popular?modeling={}&count={}'.format(model, return_count)
    
    # New Items - Returns the catalog items ranked by release date, newest first.
    elif command == 'new':
        uri = ir_base_uri + ir_path + 'new?modeling={}&count={}'.format(model, return_count)

    # Trending Items - Returns the catalog items ranked by popularity, giving newer events more weight than older ones. Popularity is scored over a configurable period of time.      
    elif command == 'trending':
        uri = ir_base_uri + ir_path + 'trending?modeling={}&count={}'.format(model, return_count)
    
    # User Picks - Returns personal picks, or items from the catalog that the user is likely to interact with.
    elif command == 'picks':
        #algo types for user picks is only RecentPurchases
        uri = ir_base_uri + ir_path + 'picks?AlgoType=RecentPurchases&modeling={}&userId={}&count={}'.format(model, seed_user, return_count)
    
    # Next Best Action - Cart or frequently bought together recommendations return items that are likely to be consumed together with the seed item(s).
    #  returns a ranked list of "next best" based on a single item
    elif command == 'fbt':
        uri = ir_base_uri + ir_path + 'cart/{}?modeling={}&count={}'.format(seed_item, model, return_count)
    # returns a ranked list of "next best" based on a list/set of mulitple items
    elif command == 'cart': 
        uri = ir_base_uri + ir_path + 'cart/items?seedItemIds={}&modeling={}&count={}&skipItems={}&userId={}'.format(seed_items_list, model, return_count, skip_count, seed_user)
     
    else:
        print("No URI could be formed")

    return uri

In [10]:
def return_recommended_items(uri, token):
    api_call_headers = {'Authorization': 'Bearer ' + token}
    api_call_response = requests.get(uri, headers=api_call_headers)  

    #note the return will be string/textual values
    rec_items = pd.json_normalize(json.loads(api_call_response.text)['items'])['id']

    #for our example, our ids need to be int, edit/remove this for your use case as needed.
    rec_items_intlist = list(map(int, rec_items))
    
    return rec_items_intlist

# Top Charts

These lists in Intelligent Recommendations enable browsing a content catalog using algorithmic charts sorted by information such as total sales, sum of clicks, release date, or a combination of different metrics. The service currently supports three base types of trending lists, Popular, New, and Trending.  For more information, see [Trending Lists] (https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/trending-lists).

## Popular Items

The most popular items from the catalog are ranked by the number of interactions over a period of time.  We request the 10 most popular items then review the descriptions in our product catalog.  

In [11]:
popular_uri = recommendation_uri_builder(ir_base_uri, 'popular', model, return_count=10)
popular_items = return_recommended_items(popular_uri, token)

print("Most Popular Items")
products_df[products_df['itemId'].isin(popular_items)]

Most Popular Items


,itemId,itemDescription,earliestDate
12,13,bottled water,2014-01-01T00:00:00.000Z
30,31,citrus fruit,2014-01-01T00:00:00.000Z
102,103,other vegetables,2014-01-01T00:00:00.000Z
122,123,rolls/buns,2014-01-02T00:00:00.000Z
123,124,root vegetables,2014-01-02T00:00:00.000Z
130,131,sausage,2014-01-01T00:00:00.000Z
138,139,soda,2014-01-01T00:00:00.000Z
156,157,tropical fruit,2014-01-01T00:00:00.000Z
164,165,whole milk,2014-01-01T00:00:00.000Z
165,166,yogurt,2014-01-01T00:00:00.000Z


In [12]:
# See the count of orders (baskets) that contain the popular items
popular_baskets_df = orders_df[orders_df['itemId'].isin(popular_items)][['itemDescription', 'basketId']].groupby('itemDescription').count()
popular_baskets_df.reset_index(inplace=True)

print("Number of times Most Popular Items have been Ordered")
popular_baskets_df.columns = ['itemDescription', 'countOfOrders']
popular_baskets_df.sort_values(by=['countOfOrders'], ascending=False)


Number of times Most Popular Items have been Ordered


,itemDescription,countOfOrders
8,whole milk,2502
2,other vegetables,1898
3,rolls/buns,1716
6,soda,1514
9,yogurt,1334
4,root vegetables,1071
7,tropical fruit,1032
0,bottled water,933
5,sausage,924
1,citrus fruit,812


## Trending Items

Returns the catalog items ranked by popularity, giving newer events more weight than older ones. Popularity is scored over a configurable period of time.

In [13]:
trending_uri = recommendation_uri_builder(ir_base_uri, 'trending', model, return_count=10)
trending_items = return_recommended_items(trending_uri, token)

print("Trending Items")
products_df[products_df['itemId'].isin(trending_items)]

Trending Items


,itemId,itemDescription,earliestDate
12,13,bottled water,2014-01-01T00:00:00.000Z
30,31,citrus fruit,2014-01-01T00:00:00.000Z
102,103,other vegetables,2014-01-01T00:00:00.000Z
122,123,rolls/buns,2014-01-02T00:00:00.000Z
123,124,root vegetables,2014-01-02T00:00:00.000Z
130,131,sausage,2014-01-01T00:00:00.000Z
138,139,soda,2014-01-01T00:00:00.000Z
156,157,tropical fruit,2014-01-01T00:00:00.000Z
164,165,whole milk,2014-01-01T00:00:00.000Z
165,166,yogurt,2014-01-01T00:00:00.000Z


## New Items

Returns the catalog items ranked by release date, newest first.



In [14]:
new_uri = recommendation_uri_builder(ir_base_uri, 'new', model, return_count=15)
new_items = return_recommended_items(new_uri, token)

print("Newest Items")
products_df[products_df['itemId'].isin(new_items)]

Newest Items


,itemId,itemDescription,earliestDate
7,8,bathroom cleaner,2014-06-10T00:00:00.000Z
36,37,cooking chocolate,2014-04-29T00:00:00.000Z
38,39,cream,2014-04-20T00:00:00.000Z
55,56,flower soil/fertilizer,2014-07-12T00:00:00.000Z
57,58,frozen chicken,2014-04-20T00:00:00.000Z
60,61,frozen fruits,2014-04-20T00:00:00.000Z
66,67,hair spray,2014-04-01T00:00:00.000Z
79,80,kitchen utensil,2014-08-14T00:00:00.000Z
87,88,male cosmetics,2014-03-28T00:00:00.000Z
100,101,organic products,2014-04-12T00:00:00.000Z


# Item-to-Item Recommendations

Item-to-item recommendations leverage user interactions with different items to learn about contextual relationships. Item-to-item relationships are one of the most important scenarios that recommendations can serve, because they power experiences such as Shopping by alternatives, continuous engagement, and discovery and exploration.  For more details, see [Item based lists] (https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/contextual-lists).

## Next Best Action - Frequently Bought Together
Returns items that are frequently bought along with a single seed item.


In [15]:
item_seed_id = 16 
print("Select one item in our cart")
products_df[products_df['itemId']==item_seed_id]

Select one item in our cart


,itemId,itemDescription,earliestDate
15,16,butter,2014-01-01T00:00:00.000Z


In [23]:
item_count = 5
fbt_picks_uri = recommendation_uri_builder(ir_base_uri, 'fbt', model, seed_item = item_seed_id, return_count = item_count)
fbt_pick_items = return_recommended_items(fbt_picks_uri, token)

print(item_count, "recommended Next Best Items")
products_df[products_df['itemId'].isin(fbt_pick_items)]

5 recommended Next Best Items


,itemId,itemDescription,earliestDate
11,12,bottled beer,2014-01-01T00:00:00.000Z
19,20,candy,2014-01-02T00:00:00.000Z
64,65,fruit/vegetable juice,2014-01-02T00:00:00.000Z
131,132,seasonal products,2014-01-09T00:00:00.000Z
141,142,soups,2014-01-04T00:00:00.000Z


## Next Best - Cart Recommendations

Similar to frequently bought together, but based off a set/list of items (e.g. more than 1 item in their cart).



In [17]:
# NOTE: no spaces in the string of item ids 
baking_items = '16,37,50'  #butter, cooking chocolate, eggs

# The seed items 
print('Several items we have in our cart so far:')
int_list = [int(s) for s in baking_items.split(',')]
products_df[products_df['itemId'].isin(int_list)]

Several items we have in our cart so far:


,itemId,itemDescription,earliestDate
15,16,butter,2014-01-01T00:00:00.000Z
36,37,cooking chocolate,2014-04-29T00:00:00.000Z
49,50,domestic eggs,2014-01-01T00:00:00.000Z


We've got a baking theme going - what other items should we buy?

In [18]:
item_count = 5
cart_picks_uri = recommendation_uri_builder(ir_base_uri, 'cart', model, seed_items_list = baking_items, return_count = item_count)
cart_pick_items = return_recommended_items(cart_picks_uri, token)

#Recommendations
print(item_count, "Other items recommended.")
products_df[products_df['itemId'].isin(cart_pick_items)]

5 Other items recommended.


,itemId,itemDescription,earliestDate
19,20,candy,2014-01-02T00:00:00.000Z
90,91,meat,2014-01-05T00:00:00.000Z
141,142,soups,2014-01-04T00:00:00.000Z
143,144,specialty bar,2014-01-07T00:00:00.000Z
164,165,whole milk,2014-01-01T00:00:00.000Z


Candy still makes sense, this does add speciality bars, whole milk which would likely also be used in baking.  Meat and soups aren't related to baking, but those are frequently bought with these other items.

## Similar Items

Returns a list of item-to-item recommendations. Recommended items can be similar to the seed item or related in some way, based on the way data is structured and the algorithm.  

**Note:** This option is only available if you selected the "Premium" feature set for your modeling component.  You could select Textual or Visual similarity (if your data includes images).

Because we only have have product titles in the example dataset, these results are likely not as meaningful as they would be if we had a textual description (or images) available.  We will use the Textual similarity variation to demonstrate the API.  

In [19]:
similar_seed_id = 21  # canned beer
print("Original item")
products_df[products_df['itemId']==similar_seed_id]

Original item


,itemId,itemDescription,earliestDate
20,21,canned beer,2014-01-02T00:00:00.000Z


In [20]:
item_count = 5
similar_uri = recommendation_uri_builder(ir_base_uri, 'similar_text', model, seed_item = similar_seed_id, return_count = item_count)
similar_items = return_recommended_items(similar_uri, token)

print(item_count, 'Similar items (based on text of title):')
products_df[products_df['itemId'].isin(similar_items)]

5 Similar items (based on text of title):


,itemId,itemDescription,earliestDate
11,12,bottled beer,2014-01-01T00:00:00.000Z
21,22,canned fish,2014-01-19T00:00:00.000Z
22,23,canned fruit,2014-01-16T00:00:00.000Z
23,24,canned vegetables,2014-01-23T00:00:00.000Z
103,104,packaged fruit/vegetables,2014-01-18T00:00:00.000Z


We see a lot of canned items, as well as another type of beer, showing the similarity based on text of titles.  However the usefulness of this information would be improved with more textual data available (product descriptions, categories, ingredients).  Also, it could be be helfpul to add filters/categories to your product catalog and use those to tailor the similarities further.  For more information on using these in Intelligent Recommendations see [Fine Tune Results](https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/fine-tune-results).

# User-to-Item Recommendations


User-to-item recommendations offer personalized suggestions based on similarities between items selected from a full user history, recent activity, or session activity.  For more details, see [Personalized Recommendations] (https://learn.microsoft.com/en-us/industry/retail/intelligent-recommendations/personalized-recommendations).

## User Picks
Returns personal picks, or items from the catalog that the user is likely to interact with based on their historical picks.

In [21]:
user_seed_id = 1000 
print('User', user_seed_id, 'has purchased these items in the past:')
orders_df[orders_df['Member_number'] == user_seed_id]

User 1000 has purchased these items in the past:


,basketId,Member_number,Date,itemId,itemDescription
0,1,1000,2014-06-24T00:00:00.000Z,106,pastry
1,1,1000,2014-06-24T00:00:00.000Z,129,salty snack
2,1,1000,2014-06-24T00:00:00.000Z,165,whole milk
3,2,1000,2015-03-15T00:00:00.000Z,131,sausage
4,2,1000,2015-03-15T00:00:00.000Z,133,semi-finished bread
5,2,1000,2015-03-15T00:00:00.000Z,165,whole milk
6,2,1000,2015-03-15T00:00:00.000Z,166,yogurt
7,3,1000,2015-05-27T00:00:00.000Z,109,pickled vegetables
8,3,1000,2015-05-27T00:00:00.000Z,139,soda
9,4,1000,2015-07-24T00:00:00.000Z,21,canned beer


In [22]:
item_count = 5
user_picks_uri = recommendation_uri_builder(ir_base_uri, 'picks', model, seed_user = user_seed_id, return_count = item_count)
user_pick_items = return_recommended_items(user_picks_uri, token)

print(item_count, 'more items to suggest based on their personal history of purchases.')
products_df[products_df['itemId'].isin(user_pick_items)]

5 more items to suggest based on their personal history of purchases.


,itemId,itemDescription,earliestDate
23,24,canned vegetables,2014-01-23T00:00:00.000Z
71,72,honey,2014-01-03T00:00:00.000Z
102,103,other vegetables,2014-01-01T00:00:00.000Z
125,126,rum,2014-01-06T00:00:00.000Z
138,139,soda,2014-01-01T00:00:00.000Z
